# Advanced Modeling and Simulation Course
### Instructor: Dr. M. Nedim ALPDEMİR

This material is prepared as part of the EE592-Advanced Modeling And Simulation course given at Yıldırım Beyazıd University (YBÜ).


## PythonPDEVS Examples

* PythonPDEVS belongs to the Parallel DEVS family of simulation modelling languages, and is grafted on the Python programming language. 

* While Parallel DEVS is the default formalism used, it is possible to use Classic DEVS and Dynamic Structure DEVS (for both Classic and Parallel DEVS) too. 

* PythonPDEVS fills the gap which currently separates efficient, but low-level, simulation tools from the high-level, but inefficient, simulation tools. This makes PythonPDEVS ideal for people who want a lightweight and efficient DEVS simulation kernel, with low turnaround times, while still having access to functionality commonly only found in heavyweight tools [1].

*1.Van Tendeloo, Y., and H. Vangheluwe. 2016. “An Overview of PythonPDEVS”. In JDF 2016 – Les Journees ´DEVS Francophones – Theorie et Applications ´ , edited by C. W. RED, 59–66.* 


In [7]:
from pypdevs.infinity import *
from pypdevs.DEVS import *
import time
from pypdevs.simulator import Simulator


class Generator(AtomicDEVS):
    def __init__(self):
        AtomicDEVS.__init__(self, "Generator")
        self.state = True
        self.outport = self.addOutPort("outport")

    def timeAdvance(self):
        if self.state:
            return 1.0
        else:
            return INFINITY

    def outputFnc(self):
        # Our message is simply the integer 5, though this could be anything
        return {self.outport: 5}

    def intTransition(self):
        self.state = False
        return self.state

In [4]:
class Queue(AtomicDEVS):
  def __init__(self):
      AtomicDEVS.__init__(self, "Queue")
      self.state = None
      self.processing_time = 1.0
      self.inport = self.addInPort("input")
      self.outport = self.addOutPort("output")

  def timeAdvance(self):
      if self.state is None:
          return INFINITY
      else:
          return self.processing_time

  def outputFnc(self):
      return {self.outport: self.state}

  def extTransition(self, inputs):
      self.state = inputs[self.inport]
      return self.state

  def intTransition(self):
      self.state = None
      return self.state

In [5]:
class CQueue(CoupledDEVS):
    def __init__(self):
        CoupledDEVS.__init__(self, "CQueue")
        self.generator = self.addSubModel(Generator())
        self.queue = self.addSubModel(Queue())
        self.connectPorts(self.generator.outport, self.queue.inport)

In [8]:
model = CQueue()
sim = Simulator(model)
sim.setVerbose(None)
# Required to set Classic DEVS, as we simulate in Parallel DEVS otherwise
sim.setClassicDEVS()
sim.setTerminationTime(5.0)
sim.simulate()


__  Current Time:       0.00 __________________________________________ 


	INITIAL CONDITIONS in model <CQueue.Generator>
		Initial State: True
		Next scheduled internal transition at time 1.00


	INITIAL CONDITIONS in model <CQueue.Queue>
		Initial State: None
		Next scheduled internal transition at time inf


__  Current Time:       1.00 __________________________________________ 


	EXTERNAL TRANSITION in model <CQueue.Queue>
		Input Port Configuration:
			port <input>:
				5
		New State: 5
		Next scheduled internal transition at time 2.00


	INTERNAL TRANSITION in model <CQueue.Generator>
		New State: False
		Output Port Configuration:
			port <outport>:
				5
		Next scheduled internal transition at time inf


__  Current Time:       2.00 __________________________________________ 


	INTERNAL TRANSITION in model <CQueue.Queue>
		New State: None
		Output Port Configuration:
			port <output>:
				5
		Next scheduled internal transition at time inf



# Example 2: Hairdresser

![Hairdresser-example](DevsExample-1.png)

## Problem Description

A Haridresser Shop is to be designed. The following design requirements are given:

* Customers arrive at the shop with mean interarrival times of 5 min. So arrival times are to be modelled by exponential distribution.
* Customers can have 3 types of service. 1 - Haircut, 2 - Beard Shave, 3 - Both These are determined randomly from a uniform distribution.
* Each haridresser has a different service time given by a min, max, and mode (5,12, 8) value. So service times are to be generated from a triangular distribution.
* Once the service time of a hairdresser is determined, this service time is multiplied by a factor depending on the service type given above with choices (1:1.5, 2:1, 3:2) (i.e. if the type is haircut, multiply the service time by 1.5 etc.) 
* simulate for 100 customers and collect statistics


In [1]:
"""
Created on Tue Jan 16 11:52:57 2018

@author: nedima
"""
class RequestTypeEnum:
    def __init__(self):
        self.requestType = ("HAIR","BEARD","HAIR_AND_BEARD")
        
class Customer:
    def __init__(self, requestType, creation_time):
        # Customers have a request type and creation_time parameter
        self.requestType = requestType
        self.creation_time = creation_time
        self.processing_time = 0


In [14]:
from pypdevs.DEVS import AtomicDEVS
#from Customer import Customer, RequestTypeEnum
import math
import random
import numpy as np

# Define the state of the processor as a structured object
class HairdresserState(object):
    def __init__(self):
        # State only contains the current event
        self.evt = None
        self.cust_creation_time = 0.0
        self.cust_processing_time = 0.0
        self.hair_count = 0
        self.beard_count = 0
        self.H_B_count = 0
        self.avg_cut_time = 0
        self.total_cut_time = 0

class HairDresser(AtomicDEVS):
    def __init__(self, nr):
        AtomicDEVS.__init__(self, "HairDresser_%i" % nr)

        self.state = HairdresserState()
        self.min = 5
        self.max = 12
        self.mode = 8
        self.max_min = self.max-self.min
        self.mode_min = self.mode - self.min
        self.F_C_RandomVariateDecider = self.mode_min/self.max_min
        self.in_customer = self.addInPort("in_customer_event")
        self.out_proc = self.addOutPort("out_proc")
        self.out_finished = self.addOutPort("out_finished")

        # Define the parameters of the model
        # self.speed = proc_param
        self.nr = nr

    def intTransition(self):
        # Just clear processing event
        self.state.evt = None
        return self.state

    def extTransition(self, inputs):
        # Received a new event, so start processing it
        self.state.evt = inputs[self.in_customer]
        # Calculate how long it will be processed
        waitInterval = np.random.triangular(self.min, self.mode, self.max)
        try:
            req_type = self.state.evt.requestType
            choices = {RequestTypeEnum().requestType[0]:1.5,RequestTypeEnum().requestType[1]:1,RequestTypeEnum().requestType[2]:2}
            factor = choices.get(req_type)
            waitInterval = waitInterval * factor
            self.state.total_cut_time = self.state.total_cut_time + waitInterval
            if req_type == RequestTypeEnum().requestType[0]:
                self.state.hair_count += 1
            elif req_type == RequestTypeEnum().requestType[1]:
                self.state.beard_count += 1
            elif req_type == RequestTypeEnum().requestType[2]:
                self.state.H_B_count += 1
        except:
            print("Exception when handling the request type of Customer Event: ",req_type)
            raise
        self.state.cust_creation_time = self.state.evt.creation_time
        self.state.cust_processing_time = waitInterval
        return self.state

    def timeAdvance(self):
        if self.state.evt:
            # Currently processing, so wait for that
            return self.state.cust_processing_time
        else:
            # Idle, so don't do anything
            return float('inf')

    def outputFnc(self):
        # Output the processed event and signal as finished
        return {self.out_proc: self.state,
                self.out_finished: self.nr}


In [25]:
"""
Created on Tue Jan 16 11:38:42 2018

@author: nedima
"""

from pypdevs.DEVS import AtomicDEVS
#from Customer import Customer, RequestTypeEnum
import random

# Define the state of the generator as a structured object
class GeneratorState:
    def __init__(self, gen_num):
        # Current simulation time (statistics)
        self.current_time = 0.0
        # Remaining time until generation of new event
        self.remaining = 0.0
        # Counter on how many events to generate still
        self.to_generate = gen_num

class Generator(AtomicDEVS):
    def __init__(self, gen_param, gen_num):
        AtomicDEVS.__init__(self, "Generator")
        # Output port for the event
        self.out_event = self.addOutPort("out_customer_event")
        # Define the state
        self.state = GeneratorState(gen_num)

        # Parameters defining the generator's behaviour
        self.gen_param = gen_param
        #self.size_param = size_param

    def intTransition(self):
        # Update simulation time
        self.state.current_time += self.timeAdvance()
        # Update number of generated events
        self.state.to_generate -= 1
        if self.state.to_generate == 0:
            # Already generated enough events, so stop
            self.state.remaining = float('inf')
        else:
            # Still have to generate events, so sample for new duration
            # recall that state.remining tells us the time interval for the next customer
            self.state.remaining = random.expovariate(self.gen_param)
        return self.state

    def timeAdvance(self):
        # Return remaining time; infinity when generated enough
        return self.state.remaining

    def outputFnc(self):
        # create a random request type.
        # choose from among 3 choices with pre-defined probabilities 
        choice = np.random.choice(3, p=[0.5, 0.3, 0.2])
        request_type = RequestTypeEnum().requestType[choice]
        # Calculate current time (note the addition!)
        creation = self.state.current_time + self.state.remaining
        # Output the new event on the output port
        return {self.out_event: Customer(request_type, creation)}

In [26]:
"""
Created on Tue Jan 16 11:52:45 2018

@author: nedima
"""

from pypdevs.DEVS import AtomicDEVS
import math

# Define the state of the queue as a structured object
class QueueState:
    def __init__(self, hairdresser_num):
        # Keep a list of all idle hairderessers
        self.idle_procs = list(range(hairdresser_num))
        # Keep a list that is the actual queue data structure
        self.queue = []
        # Keep the process that is currently being processed
        self.processing = None
        # Time remaining for this event
        self.remaining_time = float("inf")
        #record the max queue size
        self.max_queue_size = 0
        

class CustomerQueue(AtomicDEVS):
    def __init__(self, hairdresser_num):
        AtomicDEVS.__init__(self, "CustomerQueue")
        # Fix the time needed to process a single event
        self.processing_time = 1.0
        self.state = QueueState(hairdresser_num)

        # Create 'outputs' output ports
        # 'outputs' is a structural parameter!
        self.out_proc = []
        for i in range(hairdresser_num):
            self.out_proc.append(self.addOutPort("hairdresser_%i" % i))

        # Add the other ports: incoming customer events and finished event
        self.in_customer_event = self.addInPort("in_customer")
        self.in_finish = self.addInPort("in_finish")

    def intTransition(self):
        # Is only called when we are outputting an event
        # Pop the first idle processor and clear processing event
        self.state.idle_procs.pop(0)
        if self.state.queue and self.state.idle_procs:
            # There are still queued elements, so continue
            self.state.max_queue_size = max(self.state.max_queue_size,len(self.state.queue))
            self.state.processing = self.state.queue.pop(0)
            self.state.remaining_time = self.processing_time
        else:
            # No events left to process, so become idle
            self.state.processing = None
            self.state.remaining_time = float("inf")
        return self.state

    def extTransition(self, inputs):
        # Update the remaining time of this job
        self.state.remaining_time -= self.elapsed
        # Several possibilities
        if self.in_finish in inputs:
            # Processing a "finished" event, so mark proc as idle
            self.state.idle_procs.append(inputs[self.in_finish])
            if not self.state.processing and self.state.queue:
                # Process first task in queue
                self.state.processing = self.state.queue.pop(0)
                self.state.remaining_time = self.processing_time
        elif self.in_customer_event in inputs:
            # Processing an incoming event
            if self.state.idle_procs and not self.state.processing:
                # Process when idle processors
                self.state.processing = inputs[self.in_customer_event]
                self.state.remaining_time = self.processing_time
            else:
                # No idle processors, so queue it
                self.state.queue.append(inputs[self.in_customer_event])
                self.state.max_queue_size = max(self.state.max_queue_size,len(self.state.queue))
        return self.state

    def timeAdvance(self):
        # Just return the remaining time for this event (or infinity else)
        return self.state.remaining_time

    def outputFnc(self):
        # Output the event to the processor
        port = self.out_proc[self.state.idle_procs [0]]
        return {port: self.state.processing}


In [27]:
"""
@author: nedima
"""
from pypdevs.DEVS import CoupledDEVS

class BarberShop(CoupledDEVS):
    def __init__(self, mu, num, hairdresser_num):
        CoupledDEVS.__init__(self, "BarberShop")

        # Define all atomic submodels of which there are only one
        generator = self.addSubModel(Generator(mu, num))
        customer_queue = self.addSubModel(CustomerQueue(hairdresser_num))
        transducer = self.addSubModel(Transducer())

        self.connectPorts(generator.out_event, customer_queue.in_customer_event)

        # Instantiate desired number of processors and connect
        hairdressers = []
        for i in range(hairdresser_num):
            hairdressers.append(self.addSubModel(
                              HairDresser(i)))
            self.connectPorts(customer_queue.out_proc[i],
                              hairdressers[i].in_customer)
            self.connectPorts(hairdressers[i].out_finished,
                             customer_queue.in_finish)
            self.connectPorts(hairdressers[i].out_proc,
                              transducer.in_event)

        # Make it accessible outside of our own scope
        self.transducer = transducer
        self.customer_queue = customer_queue
        self.hairdressers = hairdressers


In [28]:
from pypdevs.DEVS import AtomicDEVS

# Define the state of the collector as a structured object
class TransducerState(object):
    def __init__(self):
        # Contains received events and simulation time
        self.events = []
        self.current_time = 0.0
class SummaryStatistics(object):
    def __init__(self):
        self.avg_cust_wait_time = 0.0
        self.longest_cust_wait_time = 0.0
        self.total_hair_cut = 0
        self.total_beard_cut = 0
        self.total_H_B_cut = 0
class EventTimes(object):
    def __init__(self):
        self.creation_time = 0.0
        self.processing_time = 0.0
        self.queueing_time = 0.0
class Transducer(AtomicDEVS):
    def __init__(self):
        AtomicDEVS.__init__(self, "Transducer")
        self.state = TransducerState()
        self.sum_stats = SummaryStatistics()
        self.event_times = EventTimes()
        self.in_event = self.addInPort("in_event")
        self.total_wait_time = 0
        self.counter = 0

    def extTransition(self, inputs):
        # Update simulation time
        self.state.current_time += self.elapsed
        # Calculate time in queue
        evnt = inputs[self.in_event]
        event_times = EventTimes()
        event_times.creation_time = evnt.cust_creation_time
        event_times.processing_time = evnt.cust_processing_time
        time = self.state.current_time - evnt.cust_creation_time - evnt.cust_processing_time
        event_times.queueing_time = max(0.0, time)
        self.sum_stats.longest_cust_wait_time = max(self.sum_stats.longest_cust_wait_time, time)
        self.counter += 1
        self.total_wait_time = self.total_wait_time  + max(0.0, time)
        self.sum_stats.avg_cust_wait_time = self.total_wait_time  / self.counter
        self.sum_stats.total_beard_cut = self.sum_stats.total_beard_cut + inputs[self.in_event].hair_count
        self.sum_stats.total_hair_cut = self.sum_stats.total_hair_cut + inputs[self.in_event].beard_count
        self.sum_stats.total_H_B_cut = self.sum_stats.total_H_B_cut + inputs[self.in_event].H_B_count
        
        # Add incoming event to received events
        self.state.events.append(event_times)
        return self.state

    # Don't define anything else, as we only store events.
    # Collector has no behaviour of its own.


In [39]:

from pypdevs.simulator import Simulator
import random

# Import the model we experiment with
#from BarberShop import BarberShop

# Configuration:
# 1) number of customers to simulate
cust_num = 100
# 2) average time between two customers
time = 5.0

# 5) maximum number of hairdressers used
max_hairdressers = 6
# End of configuration

# Store all results for output to file
values = []
summary_stats = []

def collectHairdresserStats(objList):
    total_beard_cut = 0
    total_hair_cut = 0
    total_H_B_cut = 0
    total_cut_time = 0
    for x in objList:
        total_beard_cut = total_beard_cut + x.state.beard_count
        total_hair_cut = total_hair_cut + x.state.hair_count
        total_H_B_cut = total_H_B_cut + x.state.H_B_count
        total_cut_time = total_cut_time + x.state.total_cut_time
    return [total_beard_cut, total_hair_cut, total_H_B_cut, total_cut_time]

# Loop over different configurations
for i in range(1, max_hairdressers):
    # Make sure each of them simulates exactly the same workload
    random.seed(1)
    m = BarberShop(mu=1.0/time, num=cust_num, hairdresser_num=i) 
    # PythonPDEVS specific setup and configuration
    sim = Simulator(m)
    sim.setClassicDEVS()
    sim.simulate()

    # Gather information for output
    evt_list = m.transducer.state.events
    max_q_size = m.customer_queue.state.max_queue_size   
    avg_cust_wait_time = m.transducer.sum_stats.avg_cust_wait_time
    longest_cust_wait_time = m.transducer.sum_stats.longest_cust_wait_time
    t=collectHairdresserStats(m.hairdressers)
    avg_serv_time = t[3] / (t[0] + t[1] + t[2])
    values.append([e.queueing_time for e in evt_list])
    strLine = "{:14d} | {:16.3f} | {:16.3f} | {:16.3f}  | {:16.3f} | {:16.3f} | {:16.3f}".format(i, max_q_size, longest_cust_wait_time, t[0], t[1], t[2], avg_serv_time)
    summary_stats.append(strLine)

# Write data to file
with open('output.csv', 'w') as f:
    for i in range(cust_num):
        f.write("%s" % i)
        for j in range(len(values)):
            f.write(", %5f" % (values[j][i]))
        f.write("\n")
        
    f.write("\n")
    header1 = " {:14s}|{:18s}|{:18s}|{:19s}|{:18s}|{:18s}|{:18s}\n".format("hairdrsr count","max queue size","longest wait time","total beardcut","total haircut","total H&B","avg service Time")
    header2 = " {0:14s}|{1:18s}|{1:18s}|{1:19s}|{1:18s}|{1:18s}|{1:18s}\n".format("--------------","------------------")
    f.write(header2)
    f.write(header1)
    f.write(header2)
    for i in range(len(summary_stats)):
        f.write("%s" % (summary_stats[i]))
        f.write("\n")
